---
title: "Introduction to Pandas"
author: "Vahram Poghosyan"
date: "2023-01-13"
categories: ["Python", "Python Scientific Libraries", "Pandas"]
image: "conways_game_of_life_first_scala_project.png"
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Importing Pandas

[Pandas](https://pandas.pydata.org/) is a library that contains pre-written code to help wrangle with data. We can think of it as Python's equivalent of Excel. 

We import Pandas into our development environment as we import any other library — using the `import` command.

```python
import pandas
```
It's standard to import Pandas with the shorthand `pd` in order to avoid typing `pandas` all the time.

```python
import pandas as pd
```

This gives us access to a vast array of pre-built objects, functions, and methods which are detailed in the [API reference](https://pandas.pydata.org/docs/reference/index.html#api).


# Two Underlying Data Types

## Series
The basic unit of Pandas is the `pandas.Series` object which, in keeping with the Excel analogy, can be thought of as a column in an Excel table. It's a one-dimensional data structure that's derived from a [NumPy](https://numpy.org/) array. However, unlike a NumPy array, the indices of a `Series` object aren't limited to the integer values $0,1,...n$ — they can also be descriptive labels. 

Let's create a `Series` object representing the populations of the G-7 countries in units of millions.

In [3]:
import pandas as pd
g7_pop = pd.Series([35,63,80,60,127,64,318])

As we can see, creating a series is a matter of passing a Python list (or a Numpy array) into the `Series` constructor.

### Indexing
Indexing a `Series` object is similar to indexing a Python list. For instance, let's print the first element in the above series.

In [57]:
g7_pop[0]

35

Let's now swap out the integer-based indices with descriptive labels. Each `Series` object has an `index` argument that can be overwritten. 

In [58]:
g7_pop.index = [
    'Canada',
    'France',
    'Germany',
    'Italy',
    'Japan',
    'UK',
    'US'
]

Now, we can print the first element of the series using its descriptive label.

In [59]:
g7_pop['Canada']

35

We may notice a similarity between a standard Python dictionary and a labeled `Series` object. Namely, indexing a series with a label and keying into a Python dictionary have similar syntax. In fact, it's possible to create a labeled `Series` object directly from a Python dictionary. 

In [79]:
g7_pop = pd.Series({
    'Canada' : 35,
    'France' : 63,
    'Germany' : 80,
    'Italy' : 60,
    'Japan' : 127,
    'UK' : 64,
    'US' : 318
})
g7_pop

Canada      35
France      63
Germany     80
Italy       60
Japan      127
UK          64
US         318
dtype: int64

In the event of overriding the integer-based indices, it's still possible to access the elements of a `Series` sequentially using the `iloc` method (short for "integer location"). To retrieve the population of Canada, we can do as follows: 

In [94]:
g7_pop.iloc[0]

35

It's also possible to use a range when indexing. For instance, suppose we'd like to retrieve the populations of the first three countries from `g7_pop`. We can simply write:

In [95]:
g7_pop['Canada':'Germany']

Canada     35
France     63
Germany    80
dtype: int64

Or equivalently:

In [99]:
g7_pop.iloc[0:3]

Canada     35
France     63
Germany    80
dtype: int64

Since the `Series` object is based on a Numpy array, it also supports *multi-indexing* through passing a list of indices or a Boolean mask.

For instance, to print the populations of Canada and Germany at the same time, we can pass in the list `['Canada','Germany']` or the Boolean mask `[True, False, True, False, False, False, False]`. 

In [61]:
g7_pop[['Canada','Germany']]

Canada     35
Germany    80
dtype: int64

In [62]:
g7_pop[[True, False, True, False, False, False, False]]

Canada     35
Germany    80
dtype: int64

### Broadcasted and Vectorized Operations

Since it's based on a NumPy array, a `Series` object also supports vectorization and broadcasted operations. 

As a quick reminder, *vectorization* is the process by which NumPy optimizes looping in Python. It stores the array internally in a contiguous block of memory and restricts its contents to only one data type. Letting Python know this data type in advance, NumPy can then skip the per-iteration type checking that Python normally does in order to speed up our code. In fact, NumPy delegates most of the operations on such optimized arrays to pre-written C code under the hood.

*Broadcasting*, on the other hand, is the optimized process by which NumPy performs arithmetic and Boolean operations on arrays of unequal dimensions. It's an overloading of arithmetic and Boolean operators.

For instance, suppose the projected population growth of each G-7 country is `10 mln` by the year 2030. Instead of looping through the `Series` object and adding `10` to each row (or using a list comprehension), we can simply use broadcasted addition.  

In [64]:
g7_2030_pop = g7_pop + 10
g7_2030_pop

Canada      45
France      73
Germany     90
Italy       70
Japan      137
UK          74
US         328
dtype: int64

### Filtering

Thanks to broadcasted Boolean operations and multi-indexing with a Boolean mask, it's possible to write concise and readable filtering expressions on `Series`.

For instance, let's return the list of countries with a population over `70 mln`.

In [65]:
g7_pop[g7_pop >= 70]

Germany     80
Japan      127
US         318
dtype: int64

The expression `g7_pop >= 70` is a broadcasted Boolean operation on the `Series` object `g7_pop` which returns a Boolean array `[False, False, True, False, True, False, True]`. Then `g7_pop` is multi-indexed using this Boolean mask.

As another example of readable filtering expressions, we can return the list of countries whose populations exceed the mean population.  

In [66]:
g7_pop[g7_pop >= g7_pop.mean()]

Japan    127
US       318
dtype: int64

## DataFrame

Each `DataFrame` is composed of one or more `Series`. Whereas a `Series` is analogous to a column of an Excel table, a `DataFrame` is analogous to the table itself.  

The `DataFrame` constructor accepts a variety of input types, among them an ndarray and a dictionary.

If we're passing an ndarray, it becomes necessary to specify the column labels separately. Additionally, we may overwrite the integer-based indexing as we did with the `Series` object. 

In [77]:
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
df = pd.DataFrame(data, index = ['R1','R2','R3'], columns = ['C1', 'C2', 'C3'])
df

,C1,C2,C3
R1,1,2,3
R2,4,5,6
R3,7,8,9


We may bypass specifying columns manually by passing in a dictionary instead:

In [88]:
data = {
    'C1' : [1, 2, 3],
    'C2' : [4, 5, 6],
    'C3' : [7, 8, 9]
}
df = pd.DataFrame(data, index = ['R1','R2','R3'])
df

,C1,C2,C3
R1,1,4,7
R2,2,5,8
R3,3,6,9


> Note: Whereas in a `Series` the keys of the input dictionary were the row labels, in a `DataFrame` they're the column labels.

In practice we often create a `DataFrame` from a CSV file using the `pandas.read_csv()` method like so:

In [ ]:
#hide-output
csv_path = 'file.csv' #Stores the path to a CSV
df = pd.read_csv(csv_path)

> Tip: We may optionally pass `header = None` as an argument to `read_csv()` after `csv_path` if the first row of the CSV file itself is a data point, and not a header.

<br>

> Tip: Pandas also supports reading an Excel file into a `DataFrame` using the `pandas.read_excel()` method.

### Common Methods

Here are the common `DataFrame` methods that we should keep in our toolbox. These methods give us an overview of our data, and help us clean it up. 

* `df.head()` shows, by default, the first 5 rows of the dataset. Accepts an integer argument for the number of rows to display.
* `df.tail()` shows the last 5 rows, and also accepts an integer argument.
* `df.info()` gives a bird's eye overview of the dataset by showing the total rows/columns, the number of non-null datapoints, and the data types.
* `df.describe()` returns statistically significant values for each column such as, the mean, standard deviation, minimum, and maximum values.
* `df.shape` - returns the dimension of the dataset as an $(m,n)$ tuple.

### Indexing

Let's add to the dataset of G-7 countries the columns `'GDP'` and `'Surface Area'`.

In [89]:
g7_df = pd.DataFrame({
    'Population' : g7_pop,
    'GDP' : [1.7, 2.8, 3.8, 2.1, 4.6, 2.9, 1.7],
    'Surface Area' : [9.0, 0.6, 0.3, 0.3, 0.3, 0.2, 9.0]
})
g7_df

,Population,GDP,Surface Area
Canada,35,1.7,9.0
France,63,2.8,0.6
Germany,80,3.8,0.3
Italy,60,2.1,0.3
Japan,127,4.6,0.3
UK,64,2.9,0.2
US,318,1.7,9.0


Whereas in a `Series`, the primary axis of indexing are the rows, in a `DataFrame` the primary axis are the columns. Thus, indexing a column works as expected.

In [92]:
g7_df['GDP']

Canada     1.7
France     2.8
Germany    3.8
Italy      2.1
Japan      4.6
UK         2.9
US         1.7
Name: GDP, dtype: float64

Multi-indexing also works in the familiar way:

In [93]:
g7_df[['Population','GDP']]

,Population,GDP
Canada,35,1.7
France,63,2.8
Germany,80,3.8
Italy,60,2.1
Japan,127,4.6
UK,64,2.9
US,318,1.7


If we want to index rows, however, we must use the `loc` or `iloc` methods. 

For instance, say we are interested in the population, GDP, and surface area of only the first three countries. We could query the dataset like so:

In [100]:
g7_df.loc['Canada':'Germany']

,Population,GDP,Surface Area
Canada,35,1.7,9.0
France,63,2.8,0.6
Germany,80,3.8,0.3


If we're only interested in the population and GDP of the first three countries, then we could instead do the following:

In [104]:
g7_df[['Population', 'GDP']].loc['Canada':'Germany']

,Population,GDP
Canada,35,1.7
France,63,2.8
Germany,80,3.8


Noting that `loc` accepts two inputs, one for the selection of rows and one for columns, we can achieve the above more concisely as follows:

In [105]:
g7_df.loc['Canada':'Germany', ['Population', 'GDP']]

,Population,GDP
Canada,35,1.7
France,63,2.8
Germany,80,3.8


### Filtering

Since the `loc` method also accepts a Boolean mask as input, we use it to filter the `DataFrame` by row. For instance, suppose we want the GDP of countries with a population over `70 mln`. We query the dataset as follows:

In [106]:
g7_df.loc[g7_df['Population'] >= 70, 'GDP']

Germany    3.8
Japan      4.6
US         1.7
Name: GDP, dtype: float64

As additional exercise, suppose we are only interested in the population and GDP of countries that are smaller than `1.0 mln` square kilometers.

In [108]:
g7_df.loc[g7_df['Surface Area'] <= 1.0, ['Population','GDP']]

,Population,GDP
France,63,2.8
Germany,80,3.8
Italy,60,2.1
Japan,127,4.6
UK,64,2.9


### Adding, Dropping, and Renaming Columns and Rows

Let's add a `'Languages'` column to `g7_df`. We simply follow the same syntax as adding a key to a dictionary... 

In [125]:
# A DataFrame row is a series, so first we define one...
languages = pd.Series({
    'Canada' : 'French, English',
    'France' : 'French',
    'Germany' : 'German',
    'Italy' : 'Italian',
    'Japan' : 'Japanese',
    'UK' : 'English',
    'US' : 'English'
})
# Next, we add the series as a column to the DataFrame
g7_df['Languages'] = languages
g7_df

,Population,GDP,Surface Area,Languages
Canada,35,1.7,9.0,"French, English"
France,63,2.8,0.6,French
Germany,80,3.8,0.3,German
Italy,60,2.1,0.3,Italian
Japan,127,4.6,0.3,Japanese
UK,64,2.9,0.2,English
US,318,1.7,9.0,English


We can also drop a column or a row using the `drop()` method. 

Let's drop the newly created `'Languages'` column. To drop a column, we specify a `columns` argument in the `drop()` method like so:

In [126]:
g7_df.drop(columns = 'Languages')

,Population,GDP,Surface Area
Canada,35,1.7,9.0
France,63,2.8,0.6
Germany,80,3.8,0.3
Italy,60,2.1,0.3
Japan,127,4.6,0.3
UK,64,2.9,0.2
US,318,1.7,9.0


The method `drop()` returns a new `DataFrame` which does not contain the unneeded column, but the original `g7_df` still contains this column. To prove this, let's print it:

In [127]:
g7_df

,Population,GDP,Surface Area,Languages
Canada,35,1.7,9.0,"French, English"
France,63,2.8,0.6,French
Germany,80,3.8,0.3,German
Italy,60,2.1,0.3,Italian
Japan,127,4.6,0.3,Japanese
UK,64,2.9,0.2,English
US,318,1.7,9.0,English


As we can see the `'Languages'` column is still there. The solution is to specify an `inplace = True` argument so that the column is dropped in-place. 

In [128]:
g7_df.drop(columns = 'Languages', inplace = True)
g7_df

,Population,GDP,Surface Area
Canada,35,1.7,9.0
France,63,2.8,0.6
Germany,80,3.8,0.3
Italy,60,2.1,0.3
Japan,127,4.6,0.3
UK,64,2.9,0.2
US,318,1.7,9.0


In order to drop rows, we specify the `index` argument instead. Suppose we want to remove Canada and  Italy from the dataset.

In [129]:
g7_df.drop(index = ['Canada', 'Italy'])

,Population,GDP,Surface Area
France,63,2.8,0.6
Germany,80,3.8,0.3
Japan,127,4.6,0.3
UK,64,2.9,0.2
US,318,1.7,9.0


It is also possible to rename a column or a row using the `rename()` method. 

Suppose we'd like to rename the columns to include the units of measurement, and suppose we'd also like to expand the UK and US to their full names.

In [130]:
g7_df.rename(
    columns = {
        'Population' : 'Population (mln)',
        'GDP' : 'GDP (USD)',
        'Surface Area' : 'Surface Area (mln sq. km)'
    },
    index = {
        'UK' : 'United Kingdom',
        'US' : 'United States'
    }
)

,Population (mln),GDP (USD),Surface Area (mln sq. km)
Canada,35,1.7,9.0
France,63,2.8,0.6
Germany,80,3.8,0.3
Italy,60,2.1,0.3
Japan,127,4.6,0.3
United Kingdom,64,2.9,0.2
United States,318,1.7,9.0


### Manipulating Columns

Those of us familiar with Excel know the *functions* feature which allows users to select specific cells or combinations of cells, perform algebraic or logical operations with their contents, and store the results in new cells. The way to do that in Pandas is, once again, through broadcasted operations. 

For instance, suppose we'd like to add a new `'GDP Per Capita'` column to the `g7_df` dataset. This is simply a matter of dividing the GDP of each country by its population. 

Using broadcasted division, the code is simply:

In [133]:
g7_df['GDP Per Capita'] = g7_df['GDP'] / g7_df['Population']
g7_df

,Population,GDP,Surface Area,GDP Per Capita
Canada,35,1.7,9.0,0.048571
France,63,2.8,0.6,0.044444
Germany,80,3.8,0.3,0.047500
Italy,60,2.1,0.3,0.035000
Japan,127,4.6,0.3,0.036220
UK,64,2.9,0.2,0.045312
US,318,1.7,9.0,0.005346


### Worked Example - Bitcoin Price Timeseries: Cleaning and Reindexing

Sometimes we may wish to use a certain column to index a `DataFrame`. For instance, if we're working with a dataset of Bitcoin prices, it would be wise to use the `'Time'` column as the index so that we can easily access the value of Bitcoin at a given time. 


> Note: Data that's indexed by time is called a [timeseries](https://en.wikipedia.org/wiki/Time_series)... 



For this example, we will retrieve the actual daily Bitcoin price history from [CoinCap API 2.0](https://docs.coincap.io/#ee30bea9-bb6b-469d-958a-d3e35d442d7a). Feel free to check out the code that fetches the data as JSON and converts it into a Pandas `DataFrame` in the collapsable code below.

In [33]:
#collapse-hide

import requests
import json

# Specifying request URL, payload, and headers
url = 'https://api.coincap.io/v2/assets/bitcoin/history?interval=d1'
payload = {}
headers = {}
# Making the request and parsing it as JSON
response = requests.request('GET', url, headers = headers, data = payload)
json_data = json.loads(response.text)['data']
# Converting the result into a DataFrame
bitcoin_df = pd.json_normalize(json_data)
# Cleanup
bitcoin_df.rename(
    columns = {
        'priceUsd' : 'priceInUSD',
        'time' : 'Time',
        'date' : 'Date'
    },
    inplace = True
)

The result of this is the following `DataFrame`:

In [34]:
bitcoin_df.head()

,priceInUSD,Time,Date
0,37480.8939504110899664,1610668800000,2021-01-15T00:00:00.000Z
1,36853.8623471143090244,1610755200000,2021-01-16T00:00:00.000Z
2,35670.6623897365179078,1610841600000,2021-01-17T00:00:00.000Z
3,36061.4760792230247237,1610928000000,2021-01-18T00:00:00.000Z
4,36868.3293610208260669,1611014400000,2021-01-19T00:00:00.000Z


Now that we have the dataset as a cleaned-up Pandas `DataFrame` called `bitcoin_df`, we can get to work.

First order of business is to re-index the dataset based on the `'Time'` column. We can set a column as index using the `set_index()` method in the following way:

In [35]:
bitcoin_df.set_index('Time', inplace=True)
bitcoin_df.index.name = None # The index column shouldn't have a name — this removes the name 'Time'
bitcoin_df.head()

,priceInUSD,Date
1610668800000,37480.8939504110899664,2021-01-15T00:00:00.000Z
1610755200000,36853.8623471143090244,2021-01-16T00:00:00.000Z
1610841600000,35670.6623897365179078,2021-01-17T00:00:00.000Z
1610928000000,36061.4760792230247237,2021-01-18T00:00:00.000Z
1611014400000,36868.3293610208260669,2021-01-19T00:00:00.000Z


As we can see the column that was previously named `'Time'` now acts as index. 

Next, we should convert the entries of the index from a Unix `timestamp` into a Python `datetime` for more clarity. While doing this, let's also convert the entries of the `'Date'` column which are currently in string format. 

In [70]:
bitcoin_df.index = pd.to_datetime(bitcoin_df.index, unit='us') # Converting index to datetime from Unix seconds
bitcoin_df['Date'] = pd.to_datetime(bitcoin_df['Date']) # Converting 'Date' to datetime from string
bitcoin_df.head()

,priceInUSD,Date
2021-01-15,37480.8939504110899664,2021-01-15 00:00:00+00:00
2021-01-16,36853.8623471143090244,2021-01-16 00:00:00+00:00
2021-01-17,35670.6623897365179078,2021-01-17 00:00:00+00:00
2021-01-18,36061.4760792230247237,2021-01-18 00:00:00+00:00
2021-01-19,36868.3293610208260669,2021-01-19 00:00:00+00:00


Now we can comfortably access the price of Bitcoin on any given day. Suppose we'd like to know its price on `2021-12-28`, the day of writing this post... We can simply do:

In [68]:
bitcoin_df.loc['2021-12-28', 'priceInUSD']

'48995.0145281203441155'

# Common Workflows

## Finding the Unique Elements in a Column

Pandas comes with the method `unique()` which can be applied to a `Series` object. 

Let's fetch some data about the planets in our solar system from the [devstronomy repository](https://github.com/devstronomy/nasa-data-scraper/blob/master/data/csv/planets.csv).

In [27]:
planets_df = pd.read_csv('https://raw.githubusercontent.com/devstronomy/nasa-data-scraper/master/data/csv/planets.csv')
planets_df

,planet,mass,diameter,density,gravity,escape_velocity,rotation_period,length_of_day,distance_from_sun,perihelion,...,orbital_period,orbital_velocity,orbital_inclination,orbital_eccentricity,obliquity_to_orbit,mean_temperature,surface_pressure,number_of_moons,has_ring_system,has_global_magnetic_field
0,Mercury,0.3300,4879,5427,3.7,4.3,1407.6,4222.6,57.9,46.0,...,88.0,47.4,7.0,0.205,0.034,167,0,0,No,Yes
1,Venus,4.8700,12104,5243,8.9,10.4,-5832.5,2802.0,108.2,107.5,...,224.7,35.0,3.4,0.007,177.400,464,92,0,No,No
2,Earth,5.9700,12756,5514,9.8,11.2,23.9,24.0,149.6,147.1,...,365.2,29.8,0.0,0.017,23.400,15,1,1,No,Yes
3,Mars,0.6420,6792,3933,3.7,5.0,24.6,24.7,227.9,206.6,...,687.0,24.1,1.9,0.094,25.200,-65,0.01,2,No,No
4,Jupiter,1898.0000,142984,1326,23.1,59.5,9.9,9.9,778.6,740.5,...,4331.0,13.1,1.3,0.049,3.100,-110,Unknown*,79,Yes,Yes
5,Saturn,568.0000,120536,687,9.0,35.5,10.7,10.7,1433.5,1352.6,...,10747.0,9.7,2.5,0.057,26.700,-140,Unknown*,62,Yes,Yes
6,Uranus,86.8000,51118,1271,8.7,21.3,-17.2,17.2,2872.5,2741.3,...,30589.0,6.8,0.8,0.046,97.800,-195,Unknown*,27,Yes,Yes
7,Neptune,102.0000,49528,1638,11.0,23.5,16.1,16.1,4495.1,4444.5,...,59800.0,5.4,1.8,0.011,28.300,-200,Unknown*,14,Yes,Yes
8,Pluto,0.0146,2370,2095,0.7,1.3,-153.3,153.3,5906.4,4436.8,...,90560.0,4.7,17.2,0.244,122.500,-225,0.00001,5,No,Unknown


If we want to find out the unique number of moons each planet has, we can simply do:

In [29]:
planets_df['number_of_moons'].unique()

array([ 0,  1,  2, 79, 62, 27, 14,  5], dtype=int64)

As we can see, the 9 planets in our solar system (counting Pluto) have 8 unique number of moons. This is because, as we can see from the dataset, Mercury and Venus both have 0 moons.

## Saving Data

After all the data manipulation, it would be useful to save the resulting dataset locally on our machine. Pandas offers us a way to do that using the `DataFrame.to_csv()` method.

Working with the `planets_df` defined above, we can narrow the dataset down to the planets which have a gravitational force that's close to that of the Earth ($9.8  \ m/s^2$).

In [44]:
earthlike_planets_df = planets_df[(planets_df['gravity'] >= 9.8 - 1) & (planets_df['gravity'] <= 9.8 + 1)]
earthlike_planets_df

,planet,mass,diameter,density,gravity,escape_velocity,rotation_period,length_of_day,distance_from_sun,perihelion,...,orbital_period,orbital_velocity,orbital_inclination,orbital_eccentricity,obliquity_to_orbit,mean_temperature,surface_pressure,number_of_moons,has_ring_system,has_global_magnetic_field
1,Venus,4.87,12104,5243,8.9,10.4,-5832.5,2802.0,108.2,107.5,...,224.7,35.0,3.4,0.007,177.4,464,92,0,No,No
2,Earth,5.97,12756,5514,9.8,11.2,23.9,24.0,149.6,147.1,...,365.2,29.8,0.0,0.017,23.4,15,1,1,No,Yes
5,Saturn,568.00,120536,687,9.0,35.5,10.7,10.7,1433.5,1352.6,...,10747.0,9.7,2.5,0.057,26.7,-140,Unknown*,62,Yes,Yes


> Tip: Pandas prefers the use of bitwise Boolean operators `&` and `|`, instead of the Python's default `and` and `or`. This is because Pandas relies on NumPy, which in turn relies on the capacity of the bitwise operators to be overloaded.

<br>

We can now save this new dataset to our desktop as follows:

In [40]:
earthlike_planets_df.to_csv('C:/Users/Vahram/Desktop/earthlike_planets.csv')